# Benchmark - Simulations -

In [ ]:
import os
import biolqm
import subprocess
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm

from colomoto.minibn import BooleanNetwork
from merrin.bnet2flexflux import bn2flexflux

## Parameters

In [ ]:
# Files
metabolic_network_file = f'data/covert/metabolic_network.xml'


In [ ]:
# Benchmark
benchmark_dir = './benchmark/results/bn_index.csv'

# Bnet
bnet_dir = './benchmark/results/bnet/*.bnet'

# Simulations
simulations_dir = './benchmark/results/simulations/'

In [ ]:
def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

## Simulations parameters

In [ ]:
regulators = {'RPO2', 'RPb', 'RPcl', 'RPh'}

constraints = """\
obj : MAX(Growth)


Tc1 0 10.5
Tc2 0 10.5
Td 0 12
Te 0 12
Tf 0 5
Th 0 5
To2 0 15
"""

initParams = {
    "biomassReaction": "Growth",
    "timeStep": "0.01",
    "nSteps": 300,
    "initBiomass": "0.01"
}

initConcentrations = {
    "Carbon1": 10,
    "Carbon2": 10,
    "Hext": 0,
    "Dext": 0,
    "Eext": 0,
    "Fext": 0,
    "Oxygen": 100,
    "Biomass": 0
}

initRegStates = {
    "RPcl": 0,
    "RPO2": 0,
    "RPb": 0,
    "RPh": 0
}

default = {
    "constraints": constraints,
    "initParams": initParams,
    "initConcentrations": initConcentrations,
    "initRegStates": initRegStates
}

simulations = {
    "fig5": default | {"initRegStates": initRegStates | {
        "RPcl": 1
    }}, 
    "fig6": default | {"initConcentrations": initConcentrations | {
        "Carbon1": 0,
        "Oxygen": 5,
        "Hext":0
    }} | {"initRegStates": initRegStates},
    "fig7": default | {"initConcentrations": initConcentrations | {
        "Carbon1": 0,
        "Hext": 2,
    }} | {"initRegStates": initRegStates | {
        "RPh": 1
    }},
    "fig8": default | {"initConcentrations": initConcentrations | {
        "Carbon1": 0,
        "Carbon2": 5,
        "Hext": 10,
    }} | {"initRegStates": initRegStates | {
        "RPh": 1
    }},
    "fig9": default | {"initConcentrations": initConcentrations | {
        "Carbon1": 1,
        "Hext": 5,
        "Fext": 0.1,
    }} | {"initRegStates": initRegStates | {
        "RPcl": 1,
        "RPh": 1
    }},
}

## Functions

In [ ]:
def write_constraints(cfg, filename):
    with open(filename, "w") as fp:
        fp.write(cfg["constraints"])

def _write_state(fp, sect, state):
    fp.write(f"{sect}\n")
    for node, value in state.items():
        fp.write(f"{node}\t{value}\n")
        
def write_initial(cfg, filename):
    with open(filename, "w") as fp:
        for sect, value in cfg["initParams"].items():
            fp.write(f"{sect}\n{value}\n\n")
        sect = "initConcentrations"
        _write_state(fp, sect, cfg[sect])
        sect = "initRegStates"
        if sect in cfg:
            fp.write("\n")
            _write_state(fp, sect, cfg[sect])

In [ ]:
def met4j_dynamicRFBA(mn, rn, cons, init, output):
    subprocess.run(["java", "-Dlog4j.configuration=",
        "-jar", "/opt/met4j-dynamicRFBA.jar",
        "-sol", "GLPK", 
        "-s", mn, "-reg", rn,
        "-cons", cons, "-init", init, "-out", output], check=True)

In [ ]:
def read_simulation(out):
    df = pd.read_csv(out, sep="\t")
    df.set_index("Time", inplace=True)
    return df

In [ ]:
def run_simulations(mn, bnet_file, rn, output_dir, output_tmpl):
    output = {}
    
    bnet = BooleanNetwork.load(bnet_file)
    bn2flexflux(bnet, rn, regulators)

    for name, cfg in tqdm(simulations.items()):
        out = os.path.join(output_dir, output_tmpl.format(name))
        cons = os.path.join(output_dir, f"constraints-{name}.txt")
        init = os.path.join(output_dir, f"initialParameters-{name}.txt")
        write_constraints(cfg, cons)
        write_initial(cfg, init)
        met4j_dynamicRFBA(mn, rn, cons, init, out)
        output[name] = read_simulation(out)
        
    return output

## Simulations

In [ ]:
benchmark_data = []

for bnet_file in sorted(glob(bnet_dir)):
    solution_name = bnet_file.split('/')[-1].split('.')[0]
    sim_path = simulations_dir + solution_name
    create_folder(sim_path)

    rn = sim_path + '/regulatory_network.sbml'
    template = 'output-{}.csv'

    run_simulations(metabolic_network_file, bnet_file, rn, sim_path, template)